In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import optuna
import plotly.express as px
import random

# Load and prepare data
DATA_PATH = 'german_credit_data_cleaned.csv'
df = pd.read_csv(DATA_PATH)
y = df['Risk_good'].values
X_full = df.drop(columns=['Risk_good'])
sensitive = df['Sex_male'].values



c:\Users\ayush\OneDrive - iiit-b\CollegeAcads\Sem6\MOML\Project\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
GLOBAL_SEED = 873
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

In [4]:
# Define the custom objective function factory
def make_custom_obj(sens_array, alpha=0.5):
    def custom_obj(preds, dmatrix):
        labels = dmatrix.get_label()
        preds_prob = 1.0 / (1.0 + np.exp(-preds))
        grad_log = preds_prob - labels
        hess_log = preds_prob * (1.0 - preds_prob)
        mask_p = (sens_array == 1)
        mask_u = (sens_array == 0)
        n_p, n_u = mask_p.sum(), mask_u.sum()
        mean_p = preds_prob[mask_p].mean() if n_p > 0 else 0
        mean_u = preds_prob[mask_u].mean() if n_u > 0 else 0
        diff = mean_p - mean_u
        sign = np.sign(diff)
        sigmoid_deriv = preds_prob * (1.0 - preds_prob)
        grad_fair = np.zeros_like(preds)
        grad_fair[mask_p] = sign * (1.0 / n_p) * sigmoid_deriv[mask_p] if n_p > 0 else 0
        grad_fair[mask_u] = -sign * (1.0 / n_u) * sigmoid_deriv[mask_u] if n_u > 0 else 0
        hess_fair = np.zeros_like(preds)
        grad = alpha * grad_log + (1 - alpha) * grad_fair
        hess = alpha * hess_log + (1 - alpha) * hess_fair
        return grad, hess
    return custom_obj

# Define the objective function for Optuna

def create_objective_function(alpha,seed=GLOBAL_SEED):
    def objective(trial):
        # Suggest hyperparameters
        # alpha = trial.suggest_float('alpha', 0.0, 1.0)
        max_depth = trial.suggest_int('max_depth', 3, 7)
        eta = trial.suggest_float('eta', 0.01, 0.3)
        subsample = trial.suggest_float('subsample', 0.6, 1.0)
        colsample_bytree = trial.suggest_float('colsample_bytree', 0.6, 1.0)

        # Initialize cross-validation
        kf = KFold(n_splits=3, shuffle=True, random_state=seed)
        acc_scores = []
        dpd_scores = []

        # Perform 3-fold cross-validation
        for train_idx, val_idx in kf.split(X_full):
            # Split data
            X_train = X_full.iloc[train_idx].drop(columns=['Sex_male']).values
            y_train = y[train_idx]
            sens_train = sensitive[train_idx]
            X_val = X_full.iloc[val_idx].drop(columns=['Sex_male']).values
            y_val = y[val_idx]
            sens_val = sensitive[val_idx]

            # Create custom objective
            train_obj = make_custom_obj(sens_train, alpha=alpha)

            # Set up DMatrix
            dtrain = xgb.DMatrix(X_train, label=y_train)
            dval = xgb.DMatrix(X_val, label=y_val)

            # Train the model
            params = {
                'max_depth': max_depth,
                'eta': eta,
                'subsample': subsample,
                'colsample_bytree': colsample_bytree,
                'verbosity': 0,
                'seed': seed + trial.number
            }
            bst = xgb.train(
                params,
                dtrain,
                num_boost_round=100,
                obj=train_obj,
                evals=[(dtrain, 'train'), (dval, 'val')],
                early_stopping_rounds=10,
                verbose_eval=False
            )

            # Predict on validation set
            pred_prob = bst.predict(dval)
            pred = (pred_prob > 0.5).astype(int)

            # Compute accuracy and DPD
            acc = accuracy_score(y_val, pred)
            dpd_val = abs(pred[sens_val == 1].mean() - pred[sens_val == 0].mean())
            acc_scores.append(acc)
            dpd_scores.append(dpd_val)

        # Compute mean metrics across folds
        mean_acc = np.mean(acc_scores)
        mean_dpd = np.mean(dpd_scores)

        # Store metrics for analysis
        trial.set_user_attr('accuracy', mean_acc)
        trial.set_user_attr('dpd', mean_dpd)
        trial.set_user_attr('params', {
            'alpha': alpha,
            'max_depth': max_depth,
            'eta': eta,
            'subsample': subsample,
            'colsample_bytree': colsample_bytree
        })

        return mean_acc, mean_dpd
    return objective


In [10]:
# Create and run Optuna study for multi-objective optimization

def test_loss(a,dpd,accuracy):
    return (1-a)*(1-dpd) + a*accuracy

def is_pareto_efficient(acc, dpd):
    n_points = len(acc)
    is_efficient = [True] * n_points
    for i in range(n_points):
        for j in range(n_points):
            if (acc[j] >= acc[i] and dpd[j] <= dpd[i]) and (acc[j] > acc[i] or dpd[j] < dpd[i]):
                is_efficient[i] = False
                break
    return is_efficient

def normalise(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)
#adaptive weights based on distance

results = []
count_alpha = 20
trials_per_alpha = 100
sampler = optuna.samplers.NSGAIISampler(seed=GLOBAL_SEED)
study = optuna.create_study(directions=['maximize', 'minimize'], sampler=sampler)
objective = create_objective_function(alpha=0, seed=GLOBAL_SEED)  
study.optimize(objective, n_trials=trials_per_alpha)
print(f"Completed study for alpha=0")
max_val = 0
for trial in study.best_trials:
    val = test_loss(0,trial.values[1],trial.values[0])
    if val > max_val:
        max_val = val
        best_trial = trial
results.append({
    'accuracy': best_trial.values[0],
    'dpd': best_trial.values[1],
    'alpha': 0,
    'params': best_trial.user_attrs['params']
    })
study = optuna.create_study(directions=['maximize', 'minimize'], sampler=sampler)
objective = create_objective_function(alpha=1, seed=GLOBAL_SEED)  
study.optimize(objective, n_trials=trials_per_alpha)
print(f"Completed study for alpha=1")
max_val = 0
for trial in study.best_trials:
    val = test_loss(1,trial.values[1],trial.values[0])
    if val > max_val:
        max_val = val
        best_trial = trial
results.append({
    'accuracy': best_trial.values[0],
    'dpd': best_trial.values[1],
    'alpha': 1,
    'params': best_trial.user_attrs['params']
})
print(results)

max_acc = max([res['accuracy'] for res in results])
max_dpd = max([res['dpd'] for res in results])
min_acc = min([res['accuracy'] for res in results])
min_dpd = min([res['dpd'] for res in results])




for j in range(count_alpha):
    print(j)
    
    results.sort(key=lambda x: x['alpha'])
    max_dist = 0
    print(f"max_acc: {max_acc}, max_dpd: {max_dpd}, min_acc: {min_acc}, min_dpd: {min_dpd}")
    for i in range(len(results)-1):
        acc_dist = (normalise(results[i]['accuracy'],min_acc,max_acc) - normalise(results[i+1]['accuracy'],min_acc,max_acc))**2
        dpd_dist = (normalise(results[i]['dpd'],min_dpd,max_dpd) - normalise(results[i+1]['dpd'],min_dpd,max_dpd))**2

        dist = acc_dist + dpd_dist
        if dist > max_dist:
            max_dist = dist
            print(f"max dist: {max_dist}, acc_dist: {acc_dist}, dpd_dist: {dpd_dist}")
            alpha = (results[i]['alpha']+results[i+1]['alpha'])/2
    study = optuna.create_study(directions=['maximize', 'minimize'], sampler=sampler)
    objective = create_objective_function(alpha=alpha, seed=GLOBAL_SEED)  
    study.optimize(objective, n_trials=trials_per_alpha, )
    # print(f"Completed study for alpha={alpha}")
    max_val = 0
    for trial in study.best_trials:
        val = test_loss(alpha,trial.values[1],trial.values[0])
        if val > max_val:
            max_val = val
            best_trial = trial
        
    results.append({
        'accuracy': best_trial.values[0],
        'dpd': best_trial.values[1],
        'alpha': alpha,
        'params': best_trial.user_attrs['params']
    })
    max_acc = max(max_acc, best_trial.values[0])
    max_dpd = max(max_dpd, best_trial.values[1])
    min_acc = min(min_acc, best_trial.values[0])
    min_dpd = min(min_dpd, best_trial.values[1])
acc = [res['accuracy'] for res in results]
dpd = [res['dpd'] for res in results]
is_efficient = is_pareto_efficient(acc, dpd)
# Filter results to only include Pareto-optimal solutions
results = [res for res, efficient in zip(results, is_efficient) if efficient]
    

# Print summary of Pareto-optimal results
print("Pareto-optimal solutions:")
for res in results:
    print(f"Params: {res['params']}, Accuracy: {res['accuracy']:.4f}, DPD: {res['dpd']:.4f}")

# Prepare data for plotting
plot_df = pd.DataFrame({
    'DPD': [res['dpd'] for res in results],
    'Accuracy': [res['accuracy'] for res in results],
    'alpha': [res['alpha'] for res in results]
})



[I 2025-05-16 19:43:45,179] A new study created in memory with name: no-name-b0d58415-a8cf-4a5d-8246-347a294d8385
[I 2025-05-16 19:43:45,423] Trial 0 finished with values: [0.5000091902473095, 0.0] and parameters: {'max_depth': 4, 'eta': 0.23427942936231477, 'subsample': 0.6945965931961444, 'colsample_bytree': 0.6527247741457783}.
[I 2025-05-16 19:43:45,576] Trial 1 finished with values: [0.5000091902473095, 0.0] and parameters: {'max_depth': 3, 'eta': 0.14502880540829954, 'subsample': 0.6831438346551075, 'colsample_bytree': 0.6650056059966459}.
[I 2025-05-16 19:43:45,700] Trial 2 finished with values: [0.5000091902473095, 0.0] and parameters: {'max_depth': 7, 'eta': 0.2189983990110477, 'subsample': 0.7281971459251811, 'colsample_bytree': 0.958988210314949}.
[I 2025-05-16 19:43:45,817] Trial 3 finished with values: [0.5000091902473095, 0.0] and parameters: {'max_depth': 3, 'eta': 0.0380102539744466, 'subsample': 0.7932399343106844, 'colsample_bytree': 0.7689704204658687}.
[I 2025-05-16

Completed study for alpha=0


[I 2025-05-16 19:44:01,817] Trial 1 finished with values: [0.7071481743573719, 0.06816872313093723] and parameters: {'max_depth': 5, 'eta': 0.26804446437070234, 'subsample': 0.7768440223325587, 'colsample_bytree': 0.9826416456778452}.
[I 2025-05-16 19:44:01,988] Trial 2 finished with values: [0.7192793008059848, 0.08476099675351052] and parameters: {'max_depth': 6, 'eta': 0.24795257897021458, 'subsample': 0.8804583954508851, 'colsample_bytree': 0.8762311758971264}.
[I 2025-05-16 19:44:02,155] Trial 3 finished with values: [0.7014303088229438, 0.06482682424145518] and parameters: {'max_depth': 5, 'eta': 0.27301840257577104, 'subsample': 0.9272235602328082, 'colsample_bytree': 0.7767344519858717}.
[I 2025-05-16 19:44:02,332] Trial 4 finished with values: [0.7257063470911335, 0.06518022546804354] and parameters: {'max_depth': 7, 'eta': 0.20836801881162217, 'subsample': 0.6775262468224649, 'colsample_bytree': 0.8672560411849988}.
[I 2025-05-16 19:44:02,481] Trial 5 finished with values: [0

Completed study for alpha=1
[{'accuracy': 0.5000091902473095, 'dpd': 0.0, 'alpha': 0, 'params': {'alpha': 0, 'max_depth': 4, 'eta': 0.23427942936231477, 'subsample': 0.6945965931961444, 'colsample_bytree': 0.6527247741457783}}, {'accuracy': 0.7385818835718202, 'dpd': 0.09117831450646376, 'alpha': 1, 'params': {'alpha': 1, 'max_depth': 7, 'eta': 0.24152465711376142, 'subsample': 0.7910317238008687, 'colsample_bytree': 0.9963228652716941}}]
0
max_acc: 0.7385818835718202, max_dpd: 0.09117831450646376, min_acc: 0.5000091902473095, min_dpd: 0.0
max dist: 2.0, acc_dist: 1.0, dpd_dist: 1.0


[I 2025-05-16 19:44:24,626] Trial 0 finished with values: [0.6942848915397647, 0.05097018711350069] and parameters: {'max_depth': 5, 'eta': 0.2325644450902913, 'subsample': 0.9584475342564492, 'colsample_bytree': 0.8941830785195612}.
[I 2025-05-16 19:44:24,904] Trial 1 finished with values: [0.7064343984829965, 0.0800585520683066] and parameters: {'max_depth': 6, 'eta': 0.24433363493605106, 'subsample': 0.7121431009479138, 'colsample_bytree': 0.8930527737865122}.
[I 2025-05-16 19:44:25,214] Trial 2 finished with values: [0.7064236765278021, 0.07365540081371169] and parameters: {'max_depth': 7, 'eta': 0.1420219505231513, 'subsample': 0.952352654957318, 'colsample_bytree': 0.9055682237318756}.
[I 2025-05-16 19:44:25,568] Trial 3 finished with values: [0.6692874188577748, 0.033315882913593796] and parameters: {'max_depth': 3, 'eta': 0.142339334176878, 'subsample': 0.8016583335849767, 'colsample_bytree': 0.651029598561798}.
[I 2025-05-16 19:44:25,797] Trial 4 finished with values: [0.66213

...

18
max_acc: 0.7464349498978352, max_dpd: 0.09117831450646376, min_acc: 0.5000091902473095, min_dpd: 0.0
max dist: 0.03664830390295402, acc_dist: 0.036545961509055556, dpd_dist: 0.00010234239389846012
max dist: 0.04260473330815814, acc_dist: 0.029313335686833496, dpd_dist: 0.01329139762132464
max dist: 0.14704660378333922, acc_dist: 0.1313202485398175, dpd_dist: 0.015726355243521725
max dist: 0.46078129378605304, acc_dist: 0.4102357791762317, dpd_dist: 0.05054551460982133


[I 2025-05-16 19:49:05,868] Trial 2 finished with values: [0.6614404793632996, 0.039160973616569235] and parameters: {'max_depth': 6, 'eta': 0.14199752183660497, 'subsample': 0.9000663106913946, 'colsample_bytree': 0.7092489312520924}.
[I 2025-05-16 19:49:05,997] Trial 3 finished with values: [0.6707195657301805, 0.02661388780052924] and parameters: {'max_depth': 7, 'eta': 0.06128206150808623, 'subsample': 0.6406550340927423, 'colsample_bytree': 0.9490796175833713}.
[I 2025-05-16 19:49:06,127] Trial 4 finished with values: [0.6707195657301805, 0.02661388780052924] and parameters: {'max_depth': 4, 'eta': 0.026179061048435537, 'subsample': 0.8012226622319802, 'colsample_bytree': 0.9677228439036216}.
[I 2025-05-16 19:49:06,224] Trial 5 finished with values: [0.6664369104839278, 0.035051666034896754] and parameters: {'max_depth': 4, 'eta': 0.2551090314590642, 'subsample': 0.9509908690648581, 'colsample_bytree': 0.631974539296369}.
[I 2025-05-16 19:49:06,338] Trial 6 finished with values: [

19
max_acc: 0.7464349498978352, max_dpd: 0.09117831450646376, min_acc: 0.5000091902473095, min_dpd: 0.0
max dist: 0.03664830390295402, acc_dist: 0.036545961509055556, dpd_dist: 0.00010234239389846012
max dist: 0.04260473330815814, acc_dist: 0.029313335686833496, dpd_dist: 0.01329139762132464
max dist: 0.14704660378333922, acc_dist: 0.1313202485398175, dpd_dist: 0.015726355243521725
max dist: 0.24835743541838198, acc_dist: 0.21239457856114027, dpd_dist: 0.0359628568572417


[I 2025-05-16 19:49:17,392] Trial 1 finished with values: [0.6707195657301805, 0.02661388780052924] and parameters: {'max_depth': 4, 'eta': 0.018269644039208013, 'subsample': 0.8937133288034471, 'colsample_bytree': 0.7157141126359479}.
[I 2025-05-16 19:49:17,515] Trial 2 finished with values: [0.6707195657301805, 0.02661388780052924] and parameters: {'max_depth': 5, 'eta': 0.07204622384390555, 'subsample': 0.6622078111239567, 'colsample_bytree': 0.9528609240209303}.
[I 2025-05-16 19:49:17,613] Trial 3 finished with values: [0.6371414654768359, 0.04012584482896309] and parameters: {'max_depth': 4, 'eta': 0.2879012349783151, 'subsample': 0.6092674096878996, 'colsample_bytree': 0.9535490649553766}.
[I 2025-05-16 19:49:17,735] Trial 4 finished with values: [0.6335756495207286, 0.025908203961690246] and parameters: {'max_depth': 5, 'eta': 0.06625694777255245, 'subsample': 0.71175122410819, 'colsample_bytree': 0.6728644295629003}.
[I 2025-05-16 19:49:17,834] Trial 5 finished with values: [0.

Pareto-optimal solutions:
Params: {'alpha': 0, 'max_depth': 4, 'eta': 0.23427942936231477, 'subsample': 0.6945965931961444, 'colsample_bytree': 0.6527247741457783}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.015625, 'max_depth': 7, 'eta': 0.09846115378619846, 'subsample': 0.8176032213384978, 'colsample_bytree': 0.8378989637123635}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.01953125, 'max_depth': 5, 'eta': 0.04200097992618926, 'subsample': 0.6616152340584983, 'colsample_bytree': 0.9233753224185217}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.021484375, 'max_depth': 6, 'eta': 0.01341722865651471, 'subsample': 0.6061950111223026, 'colsample_bytree': 0.7926316116601153}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.02197265625, 'max_depth': 3, 'eta': 0.010026783869354019, 'subsample': 0.6146170236451909, 'colsample_bytree': 0.9594608842970873}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.022216796875, 'max_depth': 6, 'eta': 0.061678436905756286, 'subsample'

In [12]:
fig = px.scatter(
    plot_df,
    x='DPD',
    y='Accuracy',
    color='alpha',
    color_continuous_scale='Viridis',
    size_max=15,
    hover_data={'alpha': True, 'DPD': True, 'Accuracy': True},
    title='Pareto Front: DPD vs. Accuracy'
)
fig.update_traces(marker=dict(line_width=0))
fig.show()